In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.display import Audio
sound_file = '../beep-06.mp3'

import globvars as gv
import utils
import utils_tSNE as utsne
import utils_preproc as upre

import os
import sys
import mne
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
import multiprocessing as mpr
import matplotlib as mpl
import time
import gc;
import scipy.signal as sig
import pandas as pd 

import numpy as np
import utils_postprocess_HPC as postp
import pymatreader
import re

data_dir = gv.data_dir
from os.path import join as pjoin

light_only = 1
#light_only = 0
ndaysBefore = None

from dateutil import parser
# start_time = parser.parse("6 Sept 2021 19:05:15")
# end_time = parser.parse("8 Sept 2021 21:21:45")
start_time = parser.parse("19 Sept 2021 01:00:15")
#start_time = parser.parse("9 Nov 2021 01:00:15")
#start_time = parser.parse("26 Nov 2021 01:00:15")
start_time = parser.parse("29 Nov 2021 01:21:45")
#start_time = parser.parse("12 Dec 2021 01:21:45")
end_time = parser.parse("30 Oct 2025 21:21:45")
#end_time = parser.parse("13 Dec 2021 11:00:00")

ndaysBefore = None
#subdir = 'nointerp'
#subdir = 'nofeatsel'
subdir = 'joint_noskip'
lookup_dir = pjoin(gv.data_dir,subdir)
recent = postp.listRecent(days=ndaysBefore, lookup_dir= lookup_dir,
                          start_time=start_time,
                                   end_time=end_time)
print(len(recent))
rawnames = []
for lf in recent:
    st = 0
    if light_only:
        if not lf.startswith('_!'):
            continue
    rawname_regex = '([S0-9,]+_(on|off){1,2})'
    if light_only:
        r = re.match('_\!_'+rawname_regex+'_.*',lf)
    else:
        r = re.match('_'+rawname_regex+'_.*',lf)
    if r is None:
        print(lf)
        continue
    cr = r.groups()[0]
    if len(cr) < 8:
        print(lf)
        continue
    rawnames += [ cr ]
rawnames = list(sorted(set(rawnames)))
print(rawnames)

import utils_postprocess_HPC as postp
prefixes = postp.listRecentPrefixes(days = ndaysBefore, light_only=light_only, 
                                    lookup_dir= lookup_dir, 
                                    custom_rawname_regex = rawname_regex, 
                                    start_time=start_time,
                                   end_time=end_time)
display(prefixes)

In [ ]:
preloaded = None
from utils_postprocess_HPC import checkPrefixCollectionConsistencty

grouping_to_check= 'merge_nothing'
it_to_check = 'basic'

r = checkPrefixCollectionConsistencty(subdir,prefixes,start_time, end_time,  
                                      grouping_to_check, it_to_check,
                                      use_main_LFP_chan=1, light_only=1, 
                                     prefixes_ignore  = None, preloaded=preloaded)
missing, preloaded = r
print(missing)
import gc; gc.collect()

In [ ]:
#prefixes = ['onlyH_act'] #, 'onlyH', 'modLFP', 'modSrc_self']

In [ ]:
# prefixes =['LFPrel_noself',
#  'LFPrel_noself_onlyBpcorr',
#  'LFPrel_noself_onlyCon',
#  'LFPrel_noself_onlyRbcorr',
#  'allb_beta_noH',
#  'allb_gamma_noH',
#  'allb_tremor_noH',
#  'modLFP',
#  'modSrc_self',
#  'onlyH',
#  'onlyH_act']

In [ ]:
prefixes_all = prefixes

In [ ]:
#prefixes = ['modSrc']

In [ ]:
display(prefixes)

In [ ]:
# prefixes = [ 'onlyH',
#  'onlyH_noCB',
#  'onlyH_noFrontalMed',
#  'onlyH_noFrontalSup',
#  'onlyH_noLFP',
#  'onlyH_noOccipitalMid',
#  'onlyH_noSensorimotor',
#  'onlyH_noTemporalMid',
#  'onlyMotorSrc',
#  'onlyRestSrc']

In [ ]:
# prefixes = [ 'LFPrel_noself_onlyCon',
#  'LFPrel_noself_onlyRbcorr',
#  'modLFP',
#  'modSrc_self',
#  'onlyCBSrc',
#  'onlyH']

In [ ]:
#TODO save orig filename so that it is easier to find

In [ ]:
subdir

In [ ]:
rawnames

In [ ]:
pwd

In [ ]:
# #label_types = ['allsep', 'trem_vs_all', 'trem_vs_quiet']
# #label_types = ['allsep']

# nraws_used_PCA = 2

# set_explicit_nraws_used_PCA = 1
# set_explicit_n_feats_PCA = 0
# set_explicit_dim_PCA = 0

# try:
#     print(output_per_raw.keys() )
# except (NameError,AttributeError) as e:
#     output_per_raw = None
labels_dict = utils.loadLabelsDict()
    
#subdir='SSS'
#%debug
sources_type = 'parcel_aal'  # or ''
#subdir =''
r = postp.collectPerformanceInfo3(None,prefixes, nraws_used='[0-9]{2,4}',                                                                                                    
                                          sources_type = sources_type, 
                                           printFilenames=1,                                 
                                            ndays_before=ndaysBefore,
                                            use_main_LFP_chan=1,
                                             subdir=subdir, remove_large_items = 1,
                                 list_only=0, allow_multi_fn_same_prefix=0,
                                 use_light_files = light_only, rawname_regex_full=1,
                                  lighter_light = 0,
                                 start_time=start_time,
                                   end_time=end_time)
#nraws_used='(10,12,20,24)'
#output_per_raw,Ximp_per_raw,gis_per_raw = r
output_per_raw,_,_ = r
print('len(output_per_raw) =', len(output_per_raw))
import gc; gc.collect()

np.savez(pjoin(gv.data_dir,subdir,'gathered.npz'), output_per_raw=output_per_raw )

Audio(filename=sound_file, autoplay=True)

import utils_postprocess as pp
tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3)

In [ ]:
z0 = [tpl[:-1] for tpl in tpll]
#rns_ord, prefs_ord, grp_ord, it_ord = list (zip(*z0  ) )
tpll_reshaped = np.array( list (zip(*z0  ) ) )

In [ ]:
from utils_postprocess_HPC import checkTupleListTableCompleteness
b,missing = checkTupleListTableCompleteness(tpll_reshaped)

outputs_filtered = postp.filterOutputs(output_per_raw, rns=rawnames , 
                    prefs=prefixes, grps = ['merge_nothing'] )
b,missing =  checkTupleListTableCompleteness(outputs_filtered)
print(missing)

# Plot confmat info

In [ ]:
revdict_lenc

In [ ]:
revdict_user = {'trem_L':0, 'notrem_L':1, 'hold_L':2, 'move_L':3}

In [ ]:
from utils_postprocess_HPC import confinfo_from_EBM,confinfo_from_XGB
#rr = confinfo_from_EBM(tpll[:2])
inds = np.where( np.array(tpll_reshaped[1] ) == 'onlyH_act' )[0]
rr = confinfo_from_XGB( [tpll[ind] for ind in inds ] )

Ms_full, matnames, matdicts, colnames, revdict_lenc, main_colind = rr
print(matnames)

#from matplotlib import rc,rcParams

# activate latex text rendering
# rc('text', usetex=False)
# #rc('axes', linewidth=2)
rc('font', weight='normal')
#rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']
# extract from small confmats
#Ms = [np.eye(3), np.eye(3)*0.5]
#colnames = ['A', 'B', 'C']
#matnames = ['mat1', 'mat2']
#main_colind = 1
#def confmat2bar(M,colnames):
# 1 vs every separately
nr = 1   #len(matnames)
    #nr = dat_pri[0].shape[0]
#nc = len(parcel_names)
nc = 1
#fig,axs = plt.subplots(nr,nc,figsize=(nc*ww,nr*hh))
#axs = axs.reshape((nr,nc))
plt.figure(figsize=(20,4))
ax = plt.gca()

Nbars = 0
show_merged = 1;   Nbars += 4
show_diag = 1;     Nbars += 3
show_off_diag = 1; Nbars += 3 * 2


Mfull_per_matname = {}

for Mi,matname in enumerate(matnames):
    Mfull = np.ones((4,4)) * np.nan
    Mfull_per_matname[matname] = Mfull
    
    M = Ms_full[Mi]
    assert len(colnames) == len(M)
    barheights = [0.] * Nbars
    barnames = len(barheights) * ['']
    ind = 0
    
    tremind = revdict_user['trem_L']
    Mfull[tremind,tremind] = M[main_colind, main_colind]
    
    if show_merged:
        barheights[ind] = M[main_colind, main_colind] * 100
        barnames[ind] =   f'{colnames[main_colind]}\n{colnames[main_colind]}'
        ind += 1
        barheights[ind] = M[main_colind+1, main_colind+1]  * 100
        barnames[ind] =   f'{colnames[main_colind+1]}\n{colnames[main_colind+1]}'
        ind += 1

        barheights[ind] = M[main_colind+1, main_colind] * 100
        barnames[ind] =   f'{colnames[main_colind+1]}\n{colnames[main_colind]}'
        ind += 1
        barheights[ind] = M[main_colind, main_colind+1]  * 100
        barnames[ind] =   f'{colnames[main_colind]}\n{colnames[main_colind+1]}'
        ind += 1

    #ax = axs[0,0]    


    Mdict = matdicts[Mi]
    for (c1,c2),perf_dict_cur_cp in Mdict.items():
        M = perf_dict_cur_cp['confmat_aver']
        #print(,M)
        print(matname,c1,c2,M)
        assert c1[:4] == 'trem'
        c2ind = revdict_user[c2]
        Mfull[tremind,c2ind] = M[0, 1]
        if show_diag:
            barheights[ind] = M[0, 0] * 100
            barnames[ind] =   f'{c1}\n{c1}\nw_{c2}'
            ind += 1
        if show_off_diag:
            barheights[ind] = M[0, 1] * 100
            barnames[ind] =   f'{c1}\n!{c1}\nw_{c2}'
            ind += 1
            
            barheights[ind] = M[1, 0] * 100
            barnames[ind] =   f'!{c1}\n{c1}\nw_{c2}'
            ind += 1
        

    print(barheights )
    ax.bar(barnames,barheights,label=matnames[Mi], alpha=0.7)
# for Mi,M in enumerate(Ms):
#     barnames = [''] * len(colnames)
#     barheights = [0.]*len(colnames)
#     for cni,colname in enumerate(colnames):
#         barnames[cni] =   f'True:{colnames[main_colind]}\nPred:{colname}' 
#         barheights[cni] = M[main_colind, cni]

#     ax.bar(barnames,barheights,label=matnames[Mi])

ax.legend()
ax.set_ylim(0,100)
ax.set_title('True <label1>\n Pred <label2>')
# for pind, parcel_name in enumerate(parcel_names):
#     perf_cur = 
#     barheights += [ perf_cur ]

figfname_full = pjoin(gv.dir_fig,subdir, 'barplot_confmat.pdf')
plt.savefig(figfname_full)
print('saved to ',figfname_full)

In [ ]:
output_per_raw['S01,S02,S04,S05,S07_on']['onlyH_act_LFPand_exclude14'].keys()

In [ ]:
k,v = zip(* list(revdict_user.items()) )

dirdict_user = dict(zip(v, [kk[:-2] for kk in k ])  )
dirdict_user

dirdict_user[revdict_user['notrem_L']] = 'quiet'

plt.figure(figsize=(4,4))
ax = plt.gca()
for Mi,matname in enumerate(matnames):
    barnames = []
    barheights = []
    Mfull = Mfull_per_matname[matname]
    for k in sorted(dirdict_user.keys()):
        barnames += [ dirdict_user[k] ]
        barheights += [Mfull[0,k] * 100 ]
    ax.bar(barnames,barheights,label=matname, alpha=0.7)
ax.set_title('Tremor predicted as')
ax.set_ylabel('% of occurence')
ax.set_xlabel('behavioral state')
ax.legend()

figfname_full = pjoin(gv.dir_fig,subdir, 'barplot_confmat_small.pdf')
plt.savefig(figfname_full)
print('saved to ',figfname_full)

In [ ]:
figfname_full

In [ ]:
# extract from full confmat
#Ms = [np.eye(3), np.eye(3)*0.5]
#colnames = ['A', 'B', 'C']
#matnames = ['mat1', 'mat2']
#main_colind = 1
#def confmat2bar(M,colnames):
# 1 vs every separately

assert len(colnames) == len(Ms[0])
nr = 1   #len(matnames)
    #nr = dat_pri[0].shape[0]
#nc = len(parcel_names)
nc = 1
#fig,axs = plt.subplots(nr,nc,figsize=(nc*ww,nr*hh))
#axs = axs.reshape((nr,nc))


#ax = axs[0,0]
ax = plt.gca()

for Mi,M in enumerate(Ms):
    barnames = [''] * len(colnames)
    barheights = [0.]*len(colnames)
    for cni,colname in enumerate(colnames):
        barnames[cni] =   f'True:{colnames[main_colind]}\nPred:{colname}' 
        barheights[cni] = M[main_colind, cni]

    ax.bar(barnames,barheights,label=matnames[Mi])

ax.legend()
ax.set_ylim(0,1)
# for pind, parcel_name in enumerate(parcel_names):
#     perf_cur = 
#     barheights += [ perf_cur ]


#plt.bar()

### Full confmat (unmerged)

In [ ]:
from utils_postprocess_HPC import filterOutputs
outputs_filtered = postp.filterOutputs(output_per_raw,prefs=['onlyH_act'], grps=['merge_nothing'])

In [ ]:
outputs_filtered['S01,S02,S04,S05,S07_on']['onlyH_act']['merge_nothing']['basic'].keys()

In [ ]:
outputs_filtered['S01,S02,S04,S05,S07_on']['']

In [ ]:
printOutputInfo(  outputs_filtered, 'prefix,grouping')

# plot confusion matrices

In [ ]:
help(np.linspace)

In [ ]:
np.linspace(0,0.3,3)

# Confmats 

In [ ]:
#%debug
plt.rcParams.update({'font.size': 15})
postp.plotConfmats(outputs_filtered, ww = 5, hh =5, keep_beh_state_sides=0, labelpad_cbar=140)

In [ ]:
[tpl[:-1] for tpl in tpll_filtered]

In [ ]:
tpll_filtered = pp.multiLevelDict2TupleList(outputs_filtered,4,3) 

In [ ]:
postp.plotConfmats()

In [ ]:
#Mfull_per_matname

In [ ]:
pp.printSizeInfo(r)

In [ ]:
r['perf_per_cp']

In [ ]:
r['hyperopt_trials'].__dict__.keys()

In [ ]:
trials = r['hyperopt_trials']

In [ ]:
trials.losses

In [ ]:
a = {'K1':1, 'K2':2}
a.update({'K1':10, 'K3':3})
print(a)

In [ ]:
trial_ind = np.argmin( trials.losses() )
best_detailed = trials.results[ trial_ind ]['effparams']
best_detailed

# Brain maps

In [ ]:
pref_LFPand

In [ ]:
prefs

In [ ]:
outputs_filtered.keys()

In [ ]:
mco = tpll[0][-1]
from utils_postprocess_HPC import computeImprovementsPerParcelGroup,plotTableInfoBrain
from utils_postprocess_HPC import filterOutputs

prefs = [pref for pref in prefixes if pref.startswith('onlyH_act_only') ]
prefs += [pref for pref in prefixes if pref.startswith('onlyH_act_exclude')]
prefs += ['onlyH_act']
outputs_filtered = postp.filterOutputs(output_per_raw,prefs=prefs, grps = ['merge_nothing'] )

infos = []
for mode in ['only','exclude']:
    #%debug
    impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond =\
        computeImprovementsPerParcelGroup(outputs_filtered, 'onlyH_act', mode = mode, inv_exclude = True)
#        return impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond

    for medcond in impr_per_medcond_per_pgn.keys():
        axs,crlb, info = plotTableInfoBrain(impr_wrt_base_per_medcond_per_pgn , medcond, mco, 
                           head_subj_ind=None, mode=mode, subdir=subdir)
        infos += [(mode,medcond,info)]
        
        
        plt.close()
        #break
    #break

In [ ]:
infoinds = [0,1]
for ind in infoinds:
    info_ = infos[ind]
    print(info_[:2])
    info = info_[2]
    d = dict( zip(info['brain_area_labels'], info['intensities']) )
    print( d['Cerebellum'] )
    
    

In [ ]:
info.keys()

# Improvement compared to LFP

In [ ]:
mco = tpll[0][-1]
pref_LFPand = [pref for pref in prefixes if pref.startswith('onlyH_act_LFPand_') ]
pref_LFPand += ['onlyH_act', 'modLFP']
outputs_filtered = postp.filterOutputs(output_per_raw,prefs=pref_LFPand, grps = ['merge_nothing'])

from utils_postprocess_HPC import computeImprovementsPerParcelGroup,plotTableInfoBrain
for mode in ['LFPand_only','LFPand_exclude']:
    #%debug
    impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond =\
        computeImprovementsPerParcelGroup(outputs_filtered, 'onlyH_act', 'modLFP',
                                          mode = mode, inv_exclude = True)
#        return impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond

    for medcond in impr_per_medcond_per_pgn.keys():
        axs,crlb, info = plotTableInfoBrain(impr_wrt_base_per_medcond_per_pgn , medcond, mco, 
                        head_subj_ind=None, mode=mode, subdir=subdir)
        
        
        plt.close()
        #break
    #break

In [ ]:
list( zip(*a) )

In [ ]:
a = [ tpl[1:-2] for tpl in tpll ]
prefs,groups = list(zip(*a) )
inds = np.where( np.array(groups) == 'merge_nothing')[0]
inds = np.where( np.array(groups) == 'merge_all_not_trem')[0]
sorted( np.array(prefs)[inds], key = lambda x:x[0] )

In [ ]:
tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3, prefix_sort = prefixes)
from utils_postprocess_HPC import printOutputInfo
printOutputInfo(output_per_raw, 'grouping', sort_dict = {'prefix':prefixes} )
printOutputInfo(output_per_raw, 'prefix',  sort_dict = {'prefix':prefixes} )

# Plot performances

In [ ]:
prefixes_desired = ['LFPrel_noself',
 'allb_beta_noH',
 'allb_gamma_noH',
 'allb_tremor_noH',
 'modLFP',
 'modSrc',
 'modSrc_self',
 'onlyH',
 'onlyH_act',
 'onlyH_act_LFPand_only0',
 'onlyH_act_LFPand_only1',
 'onlyH_act_LFPand_only10',
 'onlyH_act_LFPand_only11',
 'onlyH_act_LFPand_only12',
 'onlyH_act_LFPand_only13',
 'onlyH_act_LFPand_only14',
 'onlyH_act_LFPand_only2',
 'onlyH_act_LFPand_only3',
 'onlyH_act_LFPand_only4',
 'onlyH_act_LFPand_only5',
 'onlyH_act_LFPand_only6',
 'onlyH_act_LFPand_only7',
 'onlyH_act_LFPand_only8',
 'onlyH_act_LFPand_only9',
 'onlyH_act_only0',
 'onlyH_act_only1',
 'onlyH_act_only10',
 'onlyH_act_only11',
 'onlyH_act_only12',
 'onlyH_act_only13',
 'onlyH_act_only14',
 'onlyH_act_only15',
 'onlyH_act_only2',
 'onlyH_act_only3',
 'onlyH_act_only4',
 'onlyH_act_only5',
 'onlyH_act_only6',
 'onlyH_act_only7',
 'onlyH_act_only8',
 'onlyH_act_only9']

In [ ]:
outputs_filtered_desired = postp.filterOutputs(output_per_raw, rns=rawnames , 
                    prefs=prefixes_desired, grps = ['merge_nothing'] )

In [ ]:
import utils_postprocess as pp
tpll = pp.multiLevelDict2TupleList(outputs_filtered_desired,4,3, 
                                   prefix_sort = prefixes_desired)
#tpll [0][:4]  

mult_clf_output = tpll[0][-1]
all_thrs = mult_clf_output['feat_variance_q_thr'][-3:]
#thr0,thr1,thr2='0.6','0.75','0.9'
thr0,thr1,thr2='0.87','0.92','0.99'

all_thrs = mult_clf_output['feat_variance_q_thr'][-1:]


all_LDA =  []
all_XGB = ['all_present_features','after_VF_threshold']
all_XGB += ['all_present_features','interpret_EBM']
#all_XGB += ['all_present_features','interpret_DPEBM']

for thr_cur in all_thrs:
    all_XGB += [ f'best_PCA-derived_features_{thr_cur}']

perf_to_use_list = []
if len(all_LDA):
    for v in all_LDA[1:]:
        perf_to_use_list += [('LDA',all_LDA[0],v)]
for v in all_XGB[1:]:
    perf_to_use_list += [('XGB',all_XGB[0],v,'across_subj')]

perf_to_use_list += [('XGB','all_present_features',None,None)]
###########################################
display(perf_to_use_list)

# to_show = [('allsep','merge_nothing','basic'), ('trem_vs_all','merge_all_not_trem','basic'),
#         ('trem_vs_2class','merge_movements','basic')]
#to_show = [('trem_vs_mvt','merge_movements','trem_vs_hold&move'), 
#           ('trem_vs_all','merge_all_not_trem','basic') ]
#to_show = [('trem_vs_all','merge_all_not_trem','basic') ]
to_show = [('trem_vs_all','merge_nothing','basic') ]

#             ('trem_vs_2class','merge_movements','basic'),
#           ('trem_vs_quiet','merge_nothing','trem_vs_quiet') ]
#          ('allsep','merge_nothing','basic')]



# warnings.simplefilter('error')
# table_info_per_perf_type, table_per_perf_type = \
#     postp.prepTableInfo2(output_per_raw, prefixes=prefixes, 
#     perf_to_use_list=perf_to_use_list)

#%debug
# import warnings
# with warnings.catch_warnings():
#warnings.simplefilter('error')
table_info_per_perf_type, table_per_perf_type = \
    postp.prepTableInfo3(outputs_filtered_desired, prefixes=prefixes, 
    perf_to_use_list=perf_to_use_list, to_show=to_show)

In [ ]:
perf_to_use_list

In [ ]:
#perf_tuple = perf_to_use_list[0]
#%debug
#for perf_tuple in perf_to_use_list[:1]:
for perf_tuple in perf_to_use_list:
    print(perf_tuple)
    postp.plotTableInfos2(table_info_per_perf_type, perf_tuple=perf_tuple, 
                          output_subdir=subdir,use_recalc_perf=False, 
                          prefixes_sorted=prefixes_desired, 
                          crop_rawname=slice(-3,None), set_nice_names = True) 
    plt.close()
    import gc;gc.collect()
    #break
#postp.plotTableInfos2(table_info_per_perf_type, perf_kind='LDA', keys = None, output_subdir=''): 

In [ ]:
#mult_clf_output
postp.accessMultiLevelDict(output_per_raw,f'{rawnames[0]}/{prefixes[0]}/merge_all_not_trem/basic') . keys()

In [ ]:
perf_to_use_list

In [ ]:
# hack to get correct prefixes even for buggy runstrings

from utils_postprocess_HPC import loadRunCorresp
corresp = loadRunCorresp(tpll[-1][-1])[0]
pref_cand = [prefix for prefix in corresp.keys() if (corresp[prefix][1] == 'LFP' and \
                                                   (prefix.find('exclude') >= 0 or prefix.find('only') >= 0) ) ]
print(pref_cand)
#assert len(exclLFP) == 1
good_prefs_permod = {'msrc':[], 'LFP':[]}
good_prefs_cortex_only = []
for pref in pref_cand:
    #output_per_raw[rawnames[0]][exclLFP[0]]['merge_all_not_trem']['basic']['cmd']
    cmd = output_per_raw[rawnames[0]][pref]['merge_all_not_trem']['basic']['cmd']
    for mod in good_prefs_permod.keys():
        d = dict(cmd[0])
        if d['--mods'] == mod:
            good_prefs_permod[mod] += [pref]
        print(pref, d['--mods'], )

for mod in good_prefs_permod:
    assert len(good_prefs_permod[mod]) == 1, good_prefs_permod

    
for mod in good_prefs_permod:
    good_pref = good_prefs_permod[mod][0]
    print(mod,good_pref)
    cmd = output_per_raw[rawnames[0]][good_pref]['merge_all_not_trem']['basic']['cmd']
    display( dict(cmd[0]) )

In [ ]:
cmd = output_per_raw[rawnames[0]]['onlyH_act']['merge_all_not_trem']['basic']['cmd']
display( dict(cmd[0]) )

# Main feat subsets compare

In [ ]:
plotname_pref = 'largefeatsets'
prefix2final_name = None
prefix2final_name = {'onlyH' : 'Hjorth parameters: LFP + cortex', 
                      'onlyH_act':'Hjorth activity: LFP + cortex',
                    'allb_beta_noH':'beta band-related, w/o Hjorth',
                    'allb_gamma_noH':'gamma band-related, w/o Hjorth' }
#%debug
#TODO: disp numbers
defsp = 'special:min(sens,spec)'
score = defsp
score = 'bacc'
prefixes_final = list( prefix2final_name.keys() )
#prefixes_final = prefixes_desired
#prefixes_final = ['modLFP','modSrc_self','onlyH_act']
#for perf_tuple in [('XGB', 'all_present_features', 'interpret_EBM', 'across_subj')]:
#for perf_tuple in [('XGB', 'all_present_features', 'interpret_EBM', 'across_subj')]:
for perf_tuple in [('XGB', 'all_present_features', None, None)]:    
    print(perf_tuple)
    axs = postp.plotTableInfos_onlyBar(table_info_per_perf_type, perf_tuple=perf_tuple, 
                          output_subdir=subdir,use_recalc_perf=False, 
                          prefixes_sorted=prefixes_final, prefix2final_name=prefix2final_name,
                                 crop_rawname='last', score= score, 
                                      show_add_info='num', ww=6, hh=2) 
    axs[0,0].set_xlabel('')
    
    
#plt.title(plt.title())
scstr = ''
if score != defsp:
    scstr = score
fn_full = pjoin(gv.dir_fig,subdir,f'bars_perf_dif_subsets_{plotname_pref}_{scstr}.pdf')
plt.savefig(fn_full)

# LFP vs cortex compare bars

In [ ]:
good_prefs_permod = {'msrc': ['onlyH_act_exclude15'], 'LFP': ['onlyH_act_only15']}

In [ ]:
plotname_pref = 'noLFP'
prefix2final_name = {good_prefs_permod['LFP'][0]:'LFP', 
                     good_prefs_permod['msrc'][0]:'cortex', 
                     'onlyH_act':'LFP + cortex'}
#%debug
defsp = 'special:min(sens,spec)'
score = defsp
score = 'bacc'
prefixes_final = list( prefix2final_name.keys() )
#prefixes_final = ['modLFP','modSrc_self','onlyH_act']
#for perf_tuple in [('XGB', 'all_present_features', 'interpret_EBM', 'across_subj')]:
#for perf_tuple in [('XGB', 'all_present_features', 'interpret_EBM', 'across_subj')]:
for perf_tuple in [('XGB', 'all_present_features', None, None)]:    
    print(perf_tuple)
    axs = postp.plotTableInfos_onlyBar(table_info_per_perf_type, perf_tuple=perf_tuple, 
                          output_subdir=subdir,use_recalc_perf=False, 
                          prefixes_sorted=prefixes_final, prefix2final_name=prefix2final_name,
                                 crop_rawname='last', score= score) 
    axs[0,0].set_xlabel('')
    
    
#plt.title(plt.title())
scstr = ''
if score != defsp:
    scstr = score
fn_full = pjoin(gv.dir_fig,subdir,f'bars_perf_dif_subsets_{plotname_pref}_{scstr}.pdf')
plt.savefig(fn_full)

In [ ]:
subdir

In [ ]:
prefixes

In [ ]:
tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3)
[ (ii,tpl[:-1], len(tpll[ii][-1]['feature_names_filtered']) ) for ii,tpl in enumerate(tpll ) ]

In [ ]:
ii = 25
print(tpll[ii][:-1])
len(tpll[ii][-1]['feature_names_filtered'])

In [ ]:
len(tpll[ii][-1]['featnames_for_fit'])

In [ ]:
tpll[ii][-1]['cmd']

In [ ]:
tpll[ii][-1].keys()

# Main feature signif plots

## XGB

In [ ]:
prefixes

In [ ]:
import utils_postprocess as pp
from utils_postprocess_HPC import loadFullScores 
from matplotlib.lines import Line2D


#%debug
hh_ = None
#for prefix_cur in list(sorted( set(prefixes) - set(['all']) )):
#for prefix_cur in ['cross_freqmod_beta,gamma:HFO']:
#for prefix_cur in [ 'LFPrel_noself']:
#for prefix_cur in [ 'onlyH']:
#for prefix_cur in [ 'LFPrel_noself_onlyCon']:
#for prefix_cur in [ 'allb_beta', 'onlyH']:
#for prefix_cur in [  'allb_beta', 'allb_tremor', 'allb_gamma', 'onlyH']:
#for prefix_cur in [ 'LFPrel_noself_onlyRbcorr', 'LFPrel_noself_onlyCon', 'modSrc_self',  'onlyCBSrc', 'onlyMotorSrc']:
#for prefix_cur in [ 'LFPrel_noself_onlyBpcorr', 'LFPrel_noself']:
#for prefix_cur in [ 'all']:
#for tpl_ in [('onlyH',10), ('LFPrel_noself',25)]:
#for tpl_ in [('LFPrel_noself_onlyRbcorr',15)]:
#for tpl_ in [ ('allb_beta',15),  ('allb_gamma',15), ('allb_tremor',15) ]:
#for tpl_ in [ ('LFPrel_noself_onlyBpcorr',25)]:    
#for prefix_cur in [ 'onlyH', 'LFPrel_noself', 'allb_beta']:
#for prefix_cur in prefixes:

pref_hh_tuples = [ ('all',40)]
pref_hh_tuples = [ ('onlyH',10)]
pref_hh_tuples = [('LFPrel_noself',35) ]

pref_hh_tuples = [ ('onlyH_act',40)]
feat_subset_name = 'all'

subdir_short = 'joint_noskip'
grpit_tpl = 'merge_movements','trem_vs_hold&move'
#grpit_tpl = 'merge_movements','basic'

#subdir_short = 'joint2_noskip'

from IPython import get_ipython; ipython = get_ipython()
runstr_ = '%run -i ../run/_subrun_plot_imp_HPC.py'
ipython.magic(runstr_)
#             break+
#         break
#     break


# EBM

In [ ]:
prefixes

In [ ]:
aa = [   'onlyH',
 'onlyH_act',
 'onlyH_act_CB',
 'onlyH_act_CByFS',
 'onlyH_act_CByFSyPS',
 'onlyH_act_CByFSyPSyPI',
 'onlyH_act_CByFSyPSyPIyFM',
 'onlyH_act_CBySM',
 'onlyH_act_noCB',
 'onlyH_act_noCBnoFS',
 'onlyH_act_noCBnoFSnoPS',
 'onlyH_act_noCBnoFSnoPSnoPI',
 'onlyH_act_noCBnoFSnoPSnoPInoFM',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOI',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOM',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoA',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoAnoTI',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoAnoTInoSM',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoAnoTInoSMnoTS',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoAnoTInoSMnoTSnoFI',
 'onlyH_act_noFrontalMed',
 'onlyH_act_noFrontalSup',
 'onlyH_act_noLFP',
 'onlyH_act_noOccipitalMid',
 'onlyH_act_noParietalSup',
 'onlyH_act_noSensorimotor',
 'onlyH_act_noTemporalMid',   ]

In [ ]:
aa = [ 
 'allb_beta_noH',
 'allb_gamma_noH',
 'allb_tremor_noH']

In [ ]:
[tpl[2:-1] for tpl in tpll]

In [ ]:
subdir

In [ ]:
#subdir = 'joint_noskip'
subdir_short = subdir

pref_hh_tuples = [ ('all',40)]
EBM_feat_subsets = ['VIFsel']

pref_hh_tuples = [ ('onlyH_act_SMyOIyFSyFIyTM', 5), ('onlyH',20), ('onlyH_act',20), ('LFPrel_noself',40)]
pref_hh_tuples = [ ('onlyH_act_SMyOIyFSyFIyTM', 5)]
pref_hh_tuples = [ ('LFPrel_noself',40) ]
# pref_hh_tuples = [ ('onlyH_act_noCB',20)]
# pref_hh_tuples = [ ('onlyH_act_noCBnoFS',20)]
# pref_hh_tuples =  [(a,10) for a in aa]
#pref_hh_tuples =  [(a,40) for a in aa]
##pref_hh_tuples = [ ('modSrc',50)]
pref_hh_tuples = [ ('onlyH_act',5)]
#pref_hh_tuples = [ ('onlyH',10)]
#onlyH_act_noCBnoFS
# pref_hh_tuples = [ ('onlyH_act_noFrontalSup',20)]
# pref_hh_tuples = [ ('onlyH_act_noCBnoFSnoPSnoPI',20)]

EBM_feat_subsets = ['all']


#subdir_short = 'joint_noskip'
#grpit_tpl = 'merge_movements','basic'
grpit_tpl = 'merge_movements','trem_vs_hold&move'
grpit_tpl = 'merge_all_not_trem','basic'

import gc; gc.collect()
#%debug
from IPython import get_ipython; ipython = get_ipython()
runstr_ = '%run -i ../run/_subrun_plot_imp_EBM_HPC.py'
ipython.magic(runstr_)

In [ ]:
prefixes

In [ ]:
from datetime import datetime
fname = mult_clf_output['filename_full']
mtime = os.stat(  fname).st_mtime
datetime.fromtimestamp(mtime)

In [ ]:
fname

In [ ]:
mult_clf_output['cmd']

In [ ]:
mult_clf_output_clf_output_clf_output_clf_output     

In [ ]:
d = ['Cerebellum','ParietalSup','FrontalSup', 'ParietalInf','Angular', 'OccipitalInf', 'FrontalMed']
labels_dict['all_raw']

In [ ]:
clf_dict = mult_clf_output['featsel_per_method']['interpret_EBM']['all']
print(clf_dict.keys())

from interpret import preserve
explainer = clf_dict['explainer']
scores = clf_dict['scores']

In [ ]:
clf_dict['explainer_loc'].data()

In [ ]:
clf_dict['ebm_mergedobj']

In [ ]:
sc_noi, fns_noi, _,_ = postp.splitScoresEBM(scores,explainer.feature_names)

In [ ]:
sc_noi

In [ ]:
fn_to_observe = fns_noi[-1]
print(fn_to_observe)
preserve(explainer, fn_to_observe)

In [ ]:
mult_clf_output['feature_names_filtered']

In [ ]:
dd = 'bias/6.1694/2.4873'
np.sum( np.array(list(dd)) == '/' ) 


In [ ]:
import gc;gc.collect()

In [ ]:
common = set(mult_clf_output['VIF_truncation']['colinds_bad_VIFsel'])
#print(common)
for i,og in enumerate(outputs_grouped.items() ):
    rn = og[0][0]
    (prefix,grp,int_type), mult_clf_output = og[1]
    VIFtr = mult_clf_output['VIF_truncation']
    badinds = VIFtr['colinds_bad_VIFsel']
    print(len(badinds), badinds)
    common = common & set(badinds)
    
print(len(common),sorted(common,reverse=1) )

In [ ]:
d[(rn,prefix,grp,it)]

In [ ]:
VIFtr.keys()

In [ ]:
VIFtr['VIFsel_linreg_objs'][0].__dict__.keys()

In [ ]:
d = {}
for rn,prefix,grp,it,mult_clf_output in tpll:
    VIFtr = mult_clf_output['VIF_truncation']
    dd = dict( VIFtr.items() )
    #del dd['VIFsel_linreg_objs']
    #dd[]
    #VIFtr['VIFsel_linreg_objs'][0]
    d[(rn,prefix,grp,it)] = dd
#('S01,S02,S04,S05,S07_off', 'allb_beta', 'merge_movements', 'basic')
VIF_info_fname = pjoin(gv.data_dir,subdir_short,'VIF_info.npz')
print(VIF_info_fname)
np.savez(VIF_info_fname,VIF_info = d)

In [ ]:
# (rn_,prefix,grp,int_type,fsh,featnames_nice,label_str,\
#      scores_pre_class_curlab, feat_imp_stats ) = collect_SHAP_outs[0]


runMLvars_pso = []
for cso in collect_SHAP_outs:
    (rn_,prefix,grp,int_type,fsh,featnames_nice,label_str,\
         scores_pre_class_curlab, feat_imp_stats ) = cso 
    
    mult_clf_output = output_per_raw[rn_][prefix][grp][int_type]
    #assert (prefix,grp,int_type) == (prefix_,grp_,int_type_)    
    cmd = mult_clf_output['cmd']
    print(cmd[0])

    ####################

    s = '%run -i ../run/run_ML.py '
    for a,b in cmd[0]:
        s += f'{a} {b} '
    print(s)
    from IPython import get_ipython; ipython = get_ipython()

    #%run -i ../run/run_ML.py -r S01_on_hold,S01_on_move,S02_on_hold,S02_on_move,S04_on_hold,S04_on_move,S05_on_hold,S05_on_move,S07_on_hold,S07_on_move --param_file ML_joint_one_LFP_HPC.ini --groupings_to_use merge_movements --int_types_to_use basic --feat_types H_act,H_mob,H_compl --prefix onlyH --SLURM_job_id 114703_22 --calc_MI 0

    ipython.magic(s + ' --exit_after artif_processed --show_plots 0 --do_cleanup 0')
    anndict_per_intcat_per_rawn_out = anndict_per_intcat_per_rawn

    runMLvars = {}
    runMLvars['featnames_nice'] = featnames_nice
    runMLvars['featnames'] = featnames
    runMLvars['X_pri'] = X_pri
    runMLvars['rawnames'] = rawnames
    runMLvars['Xtimes_pri'] = Xtimes_pri
    runMLvars['wbd_pri'] = wbd_pri
    runMLvars['anndict_per_intcat_per_rawn'] = anndict_per_intcat_per_rawn
    runMLvars['new_main_side_pri'] = new_main_side_pri
    runMLvars['roi_labels'] = roi_labels
    runMLvars['srcgrouping_names_sorted'] = srcgrouping_names_sorted
    runMLvars['rawtimes_pri'] = rawtimes_pri
    
    runMLvars_pso += [runMLvars]

    rng = slice(0,10)
    
    # TODO: right now these are not correct scores (perhaps) because of the way they are averaged
    featis = np.argsort(np.abs(scores_pre_class_curlab) )
    featis = featis[::-1]
    featis = featis[rng] #np.random.randint( len(featnames),size=20)

    # anns, anns_pri, times_concat, dataset_bounds, wbd_merged = utsne.concatAnns(rawnames,
    #                                                           rawtimes_pri, crop=(crop_start,crop_end),
    #                                                           side_rev_pri = side_switch_happened_pri,
    #                                                          wbd_pri = wbd_pri, sfreq=sfreq, ret_wbd_merged=1)
    # print('times_concat end {} wbd end {}'.format(times_concat[-1] * sfreq, wbd_merged[1,-1] ) )

    # ivalis = utils.ann2ivalDict(anns)
    # ivalis_tb_indarrays_merged = \
    #     utils.getWindowIndicesFromIntervals(wbd_merged,ivalis,
    #                                         sfreq,ret_type='bins_contig',
    #                                         ret_indices_type = 'window_inds',
    #                                         nbins_total=nbins_total )

    # anndict_per_intcat = f['anndict_per_intcat'][()]
    # anndict_per_intcat_per_rawn[rawn] = anndict_per_intcat
    # bindict_per_rawn[rawn] = upre.markedIntervals2Bins(anndict_per_intcat,
    #                                                    rawtimes,sfreq,wbd=wbd)
    xlim = (-4,8)
    #xlim = None
    figname = 'hists_'+ f'{(rn_,prefix,grp,int_type,label_str,fsh)}_rng={rng}.pdf'
    print('start plotting ',figname)
    postp.plotFeatHists(rawnames,featnames_nice,featis,X_pri,Xconcat, bindict_per_rawn,
                      ivalis_tb_indarrays_merged, xlim_common = xlim,nbins=25, savefig = 0)
    
    
    plt.savefig(pjoin(gv.dir_fig, subdir_short, figname))
    plt.close()


In [ ]:
import gc; gc.collect()

In [ ]:
(prefix_,grp_,int_type_)

## get info from run_genfeats (for a fixed rn_)

In [ ]:
mainLFPchan_per_rawn = {}
for runMLvars in runMLvars_pso:
    for rawn in runMLvars['rawnames']:
        subj,medcond,task  = utils.getParamsFromRawname(rawn)
        mainLFPchan_cur = best_LFP_info[subj][f'{best_LFP_prefix},merge_movements,basic']['best_LFP']
        #print(rawn,subj,mainLFPchan_cur)
        mainLFPchan_per_rawn[rawn] = mainLFPchan_cur

In [ ]:



rns = list(output_per_raw.keys() )
genfeat_info_per_rn = {}
for rn_to_plot in rns:
    mult_clf_output = output_per_raw[rn_][prefix_][grp][int_type]
    ML_info = mult_clf_output['info']
    fn_feat = ML_info['fname_feat_full_pri']

    dat_out_pri = []
    dat_lfp_hires_out_pri = []
    times_hires_out_pri = []
    times_out_pri = []
    subfeature_order_out_pri = []
    subfeature_order_lfp_hires_out_pri = []
    extdat_out_pri = []
    rawnames_out = []
    emgdat_pri = []
    
    for i,fn_feat_cur in enumerate(fn_feat):
        ffeat = np.load(fn_feat_cur, allow_pickle=True )
        ss = ' '.join([' '.join(list(tpl)) for tpl in  ffeat['cmd'][0]] )
        s = '%run -i ../run/run_genfeats.py ' + ss + ' --exit_after prescale_data'
        print(s)
        ipython.magic(s)

        dat_out_pri              += dat_pri
        dat_lfp_hires_out_pri    += dat_lfp_hires_pri
        times_hires_out_pri      += times_hires_pri
        times_out_pri            += times_pri
        subfeature_order_out_pri += subfeature_order_pri
        subfeature_order_lfp_hires_out_pri += subfeature_order_lfp_hires_pri
        extdat_out_pri           += extdat_pri
    
    import re
    for fn in fn_feat:
        r = re.match('.*(S[0-9]+_[a-z]+_[a-z]+)_.*',fn).groups()[0]
        #rint(r)
        rawnames_out += [r]


    emgraw_pri = []
    for rawi,rawn in enumerate(ML_info['rawnames']):
        emgraw = mne.io.read_raw_fif ( pjoin(gv.data_dir,rawn + '_emg_rectconv.fif') )
        chns_cur = gv.EMG_per_hand[main_side_pri[rawi]]
        emgraw.pick_channels(chns_cur)
        emgraw.resample(256)
        #emgraw_pri += [emgraw]
        emgdat_pri += [emgraw.get_data()]
        assert emgdat_pri[-1].shape[1] == len(times_out_pri[rawi])

    from featlist import replaceMEGsrcChnamesParams
    subfeature_order_newsrcgrp_pri = ML_info.get('chnames_newsrcgrp_pri')
    if subfeature_order_newsrcgrp_pri is None:
        subfeature_order_newsrcgrp_pri = [0] * len(rawnames_out)
        for rawi in range(len(runMLvars['rawnames'])):
            subfeature_order_newsrcgrp_pri[rawi] = \
                replaceMEGsrcChnamesParams(subfeature_order_out_pri[rawi], 0,9, '.*', 0)
            
            
    d = {}
    d['dat_out_pri'] = dat_out_pri
    d['dat_lfp_hires_out_pri'] = dat_lfp_hires_out_pri
    d['times_hires_out_pri'] = times_hires_out_pri
    d['times_out_pri'] = times_out_pri
    d['subfeature_order_out_pri'] = subfeature_order_out_pri
    d['subfeature_order_lfp_hires_out_pri'] = subfeature_order_lfp_hires_out_pri
    d['extdat_out_pri'] = extdat_out_pri
    d['rawnames_out'] = rawnames_out
    d['emgdat_pri'] = emgdat_pri
    d['subfeature_order_newsrcgrp_pri'] = subfeature_order_newsrcgrp_pri
    genfeat_info_per_rn[rn_to_plot] = d

# best_LFP_info_fname = pjoin(gv.data_dir, 'best_LFP_info.json')
# with open(best_LFP_info_fname, 'r') as f:
#     best_LFP_info = json.load(f)



display(mainLFPchan_per_rawn)

In [ ]:
rn_,prefix,grp,int_type,fsh,featnames_nice_VIF,label_str,\
         scores_pre_class_curlab, feat_imp_stats

In [ ]:
# plot time traces

for collect_SHAP_outs_ind in range(len(collect_SHAP_outs)):
    #collect_SHAP_outs_ind = 0
    (rn_,prefix,grp,int_type,fsh,featnames_nice_VIF,label_str,\
         scores_pre_class_curlab, feat_imp_stats ) = collect_SHAP_outs[collect_SHAP_outs_ind]
    print(rn_)
    
    runMLvars = runMLvars_pso[collect_SHAP_outs_ind]
    d = genfeat_info_per_rn[rn_]
    
#     ##########################  just for now to avoid recalc of genfeats
#     if rn_.endswith('_off'):
#         continue
#     ###########################    
        
    featis = np.argsort(np.abs(scores_pre_class_curlab) )
    featis = featis[::-1]
    
    mult_clf_output = output_per_raw[rn_][prefix][grp][int_type]
    VIF_truncation = mult_clf_output['VIF_truncation']
    colinds_good_VIFsel = VIF_truncation['colinds_good_VIFsel']
    featnames_VIFsel = runMLvars['featnames'][colinds_good_VIFsel]
    featnames_best_LFP = runMLvars['featnames']

    from globvars import gp
    beh_states_to_shade = [f'{it}_{main_side_let}' for it in gp.int_types_basic]

    # TODO: right now these are not correct scores (perhaps) because of the way they are averaged
    #%debug
    rng = slice(0,8)
    #rng = slice(0,2)
    rng_feats = slice(0,2)
    #rng_feats = slice(0,1)
    from plots import plotFeatsAndRelDat
    
    axs = plotFeatsAndRelDat(runMLvars['rawnames'][rng], 
                             featnames_VIFsel[featis[rng_feats]], d['dat_out_pri'],
                       d['subfeature_order_out_pri'],
                    runMLvars['X_pri'],[featnames_best_LFP]*len(d['rawnames_out']),
                    d['times_out_pri'],
                    runMLvars['Xtimes_pri'],
                    d['subfeature_order_newsrcgrp_pri'], runMLvars['wbd_pri'],
                    dat_hires_pri=d['dat_lfp_hires_out_pri'],
                    chnames_all_hires_pri = d['subfeature_order_lfp_hires_out_pri'],
                    times_hires_pri=d['times_hires_out_pri'],
                    anndict_per_intcat_per_rawn=runMLvars['anndict_per_intcat_per_rawn'], 
                    sfreq=sfreq, mainLFP_per_rawn = mainLFPchan_per_rawn,
                    roi_labels=roi_labels, srcgrouping_names_sorted=['all_raw']*10,
                            main_side_let = main_side_let, ww=6,hh=3,
                            beh_states_to_shade = beh_states_to_shade, 
                             extdat_pri=d['emgdat_pri'])

    plt.tight_layout()
    plt.savefig(pjoin(gv.dir_fig,'joint',f'good_feats_dyn_{(rn_,prefix,grp,int_type,label_str,fsh)}_{rng}.pdf'))
    plt.close()

In [ ]:
len(collect_SHAP_outs)

# interpretML

In [ ]:
#for prefix_cur in prefixes:
#for prefix_cur in prefixes[:3]:
#for prefix_cur in prefixes:
for prefix in ['onlyMotorSrc']:
    prefixes_to_use = [prefix_cur]

    outputs_grouped = pp.groupOutputs(output_per_raw, prefixes_to_use,
                                      ['merge_movements'],['trem_vs_hold&move'])
    print('      ',prefix_cur)
    for og in outputs_grouped.items():
        rn = og[0]
        (prefix,grp,int_type), mult_clf_output = og[1]
        assert prefix == prefix_cur
    
        filename_fullsize = mult_clf_output['filename_full']
        from pathlib import Path
        pfsz = Path(filename_fullsize)
        filename_fullsize = pjoin(pfsz.parents[0], pfsz.name[2:])
        #    finfo = os.stat( filename_fullsize )
        #    print(finfo.st_size / (1024**2))
        f = np.load(filename_fullsize,allow_pickle=True)

        results_cur =  f['results_cur'][()]

        #results_cur.keys()

        EBM = results_cur['featsel_per_method']['interpret_EBM']        
        #scores

        clf_dict = EBM['info_per_cp'][('trem_L', 'hold_L&move_L')]
        scores = clf_dict['scores']; 
        #print(len(results_cur['feature_names_filtered']), len(EBM['feature_indices_used']) )
        #print('len(scores) = ',len(scores) )
        explainer = clf_dict['explainer']
        
        print((rn,grp,int_type), utsne.sprintfPerfs(clf_dict['perf'] ) )

        del f
        del results_cur
        gc.collect()

In [ ]:
#       LFPrel_noself
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 63.70%,97.30%,73.97%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 54.75%,93.76%,65.06%
#        LFPrel_noself_onlyBpcorr
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 53.09%,96.59%,64.78%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 49.71%,93.22%,60.30%
#        LFPrel_noself_onlyCon
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 42.19%,94.97%,53.35%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 30.51%,92.18%,41.29%
#        LFPrel_noself_onlyRbcorr
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 48.94%,97.15%,62.01%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 35.56%,92.61%,46.82%
#        allb_beta
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 64.64%,95.34%,72.04%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 55.94%,90.69%,63.34%
#        cross_freqmod_beta,gamma:HFO
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 18.93%,97.11%,29.55%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 26.17%,94.16%,37.62%
#        cross_freqmod_tremor,beta:HFO
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 40.86%,95.38%,52.56%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 35.36%,92.31%,46.41%
#        cross_freqmod_tremor,gamma:beta
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 39.69%,96.07%,52.15%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 27.67%,94.66%,39.64%
#        modLFP
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 35.31%,95.57%,47.26%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 4.50%,98.84%,8.41%
#        modSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 72.72%,97.23%,80.13%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 62.01%,93.29%,70.18%
#        modSrc_self
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 63.11%,95.55%,71.22%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 55.49%,90.39%,62.74%
#        onlyCBSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 53.38%,95.51%,63.69%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 22.85%,94.62%,33.94%
#        onlyH
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 62.04%,95.14%,69.86%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 51.43%,91.03%,60.01%
#        onlyMotorSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 74.52%,97.34%,81.46%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 63.63%,94.94%,72.74%
#        onlyRestSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 66.35%,96.35%,74.51%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 60.19%,92.75%,68.41%

In [ ]:
(rn_,prefix,grp,int_type,fsh,featnames_nice,label_str,\
     scores_pre_class_curlab, feat_imp_stats ) = collect_SHAP_outs[0]

print(rn_,prefix,grp,int_type)
mult_clf_output = output_per_raw[rn_][prefix][grp][int_type]

In [ ]:
mult_clf_output = output_per_raw['S01,S02,S04,S05,S07_off']['onlyH'][grp][int_type]

In [ ]:
mult_clf_output = output_per_raw['S01,S02,S04,S05,S07_on']['onlyMotorSrc'][grp][int_type]

In [ ]:
print(range(0,6))

In [ ]:
[4  for d in [4,5] if d in [4,5]]

In [ ]:
help(plotFeatsAndRelDat)

In [ ]:
srcgroups_dict.keys()

In [ ]:
srcCoords_fn = sind_str + '_modcoord_parcel_aal.mat'

crdf = pymr.read_mat(srcCoords_fn)

# here we do not have label 'unlabeled'
labels = crdf['labels']  #indices of sources - labels_corresp_coordance
#crdf = sio.loadmat(srcCoords_fn)
#lbls = crdf['labels'][0]
#labels = [  lbls[i][0] for i in range(len(lbls)) ]

coords = crdf['coords_Jan_actual']
srcgroups_ = crdf['point_ind_corresp']



In [ ]:
del plotFeatsAndRelDat

In [ ]:
import plots
import importlib as il; il.reload(plots)

In [ ]:
runMLvars['wbd_pri'][0]

In [ ]:
extdat_out_pri[0].shape

In [ ]:
from globvars import gp
beh_states_to_shade = [f'{it}_{main_side_let}' for it in gp.int_types_basic]

# TODO: right now these are not correct scores (perhaps) because of the way they are averaged
#%debug
rng = slice(0,8)
#rng = slice(0,2)
rng_feats = slice(0,2)
#rng_feats = slice(0,1)
from plots import plotFeatsAndRelDat
axs = plotFeatsAndRelDat(runMLvars['rawnames'][rng], 
                         featnames_VIFsel[featis[rng_feats]], dat_out_pri,
                   subfeature_order_out_pri,
                runMLvars['X_pri'],[featnames_best_LFP]*len(rawnames_out),
                times_out_pri,
                runMLvars['Xtimes_pri'],
                subfeature_order_newsrcgrp_pri, runMLvars['wbd_pri'],
                dat_hires_pri=dat_lfp_hires_out_pri,
                chnames_all_hires_pri = subfeature_order_lfp_hires_out_pri,
                times_hires_pri=times_hires_out_pri,
                anndict_per_intcat_per_rawn=runMLvars['anndict_per_intcat_per_rawn'], 
                sfreq=sfreq, mainLFP_per_rawn = mainLFPchan_per_rawn,
                roi_labels=roi_labels, srcgrouping_names_sorted=['all_raw']*10,
                        main_side_let = main_side_let, ww=6,hh=3,
                        beh_states_to_shade = beh_states_to_shade, 
                         extdat_pri=emgdat_pri)

plt.tight_layout()
plt.savefig(pjoin(gv.dir_fig,'joint',f'good_feats_dyn_{(rn_,prefix,grp,int_type,fsh)}_{rng}.pdf'))
plt.close()

In [ ]:
best_feat_name

In [ ]:
print( utsne.sprintfPerfs(clf_dict['perf'] ) )
print( clf_dict['confmat_normalized'] )

In [ ]:
np.argmax(scores)

In [ ]:
assert len( explainer.feature_names ) == len(scores)
best_feat_name = explainer.feature_names [ np.argmax(scores)]
print(best_feat_name)
sortinds = np.argsort(scores)
last_ind = np.where([ explainer.feature_names[ind].find(' x ') < 0 \
                     for ind in  sortinds])[0][-1]
last_ind = sortinds[last_ind]
best_feat_name_no_int = explainer.feature_names[last_ind]
print(best_feat_name_no_int)

In [ ]:
sortinds[-1]

In [ ]:
 len(scores), last_ind

In [ ]:
explainer.feature_names[last_ind], m[last_ind], best_feat_name_no_int.find(' x ')

In [ ]:
np.where(m)[0][-1]

In [ ]:
m = [ explainer.feature_names[ind].find(' x ') < 0 \
                     for ind in np.argsort(scores) ]
print(m[344])

In [ ]:
print( explainer.__dict__.keys() )
# print( explainer.explanation_type, explainer.name, 
#      explainer.selector, explainer.feature_types)

In [ ]:
mult_clf_output['featsel_per_method']['interpret_EBM']['all'].keys()

from interpret import preserve
explainer = clf_dict['explainer']
preserve(explainer, best_feat_name_no_int)

In [ ]:
preserve(explainer, best_feat_name)

In [ ]:
explainer.visualize()

In [ ]:
# from interpret import show
# show(explainer)  # does not work if open saved

In [ ]:
# Trying to get a better distr fit for Hjorth (no success)

In [ ]:
len(featnames_nice), len(featnames_VIFsel)

In [ ]:
np.array(featnames_nice)[featis[-2:]]

In [ ]:
featnames[featis[-2:]]

In [ ]:
roi_labels['all_raw'][30]

In [ ]:
len(featnames_nice), len(featnames)

In [ ]:
featnames_nice[-5:], featnames[-5:]

In [ ]:
pjoin(subdir,'good_feats_dyn.pdf')

In [ ]:
# from plots import plotFeatsWithEverything
# #%debug
# plotFeatsWithEverything(dat_pri, rawtimes_pri, X_pri, 
#                             Xtimes_pri, dat_lfp_hires_pri, times_hires_pri,
#                             rawnames,
#                             subfeature_order_pri, subfeature_order_newsrcgrp_pri,
#                             subfeature_order_lfp_hires_pri,
#                             anndict_per_intcat_per_rawn,
#                             featnames, wbd_pri,
#                             sfreq, raw_perband_flt_pri, raw_perband_bp_pri,
#                             scale_data_combine_type,
#                             stats_multiband_flt, stats_multiband_bp,
#                             test_plots_descr, special_chns,
#                            fband_names_inc_HFO )

In [ ]:
np.min(Xconcat), np.max(Xconcat_to_fit)

In [ ]:
dd0 = Xconcat[:,featis[0]]
#dd = dd[ dd< np.quantile(dd,0.95)]
dd = np.exp(dd0) 
#dd = 1/dd
dd = np.log( dd - np.min(dd) *.99 )
#dd = dd[ dd< np.quantile(dd,0.99)]
plt.hist(dd,bins=250,alpha=0.5,label='new');

plt.hist(dd0,bins=250,alpha=0.5,label='old');
plt.legend(loc='upper right')

In [ ]:
plt.hist(dd0,bins=250)

In [ ]:
np.min(dd)

In [ ]:
h

In [ ]:
help(dist.pdf)

In [ ]:
f

In [ ]:
import scipy.stats
qm = np.quantile(dd,0.99)
y = dd [ dd < qm]
#y = np.exp(y)
size = len(y)
x = np.arange(np.min(y),np.max(y),0.5)
#y = 0.01 -np.min(dd) + dd
#x = np.arange(0.01,55,0.5)
#y = scipy.int_(np.round_(scipy.stats.vonmises.rvs(5,size=size)*47))
h = plt.hist(y, bins=x)
binh,binloc,barcont = h

# beta, rayleigh look ok
#ten best
# dist_names = ['cauchy',
#  'dweibull',
#  'foldcauchy',
#  'dgamma',
#  't',
#  'tukeylambda',
#  'fisk',
#  'weibull_min', 'erlang']

#'loglaplace',
 
dist_names = ['alpha', 'cauchy',  'dgamma', 'dweibull', 'erlang', 'exponweib',
              'genlogistic',  'foldcauchy',  'genextreme']
#  'fisk',  'fatiguelife',  'gumbel_r',  'gengamma','invweibull',
#  'invgamma',  'mielke',  'powerlognorm',  'nct',
#  'johnsonsu',  'lognorm',  'invgauss',  'loglaplace',
#  'recipinvgauss',  'pearson3',  'laplace', 't',  'tukeylambda',
#  'hypsecant',  'weibull_min',  'logistic',  'kstwobign',
#               'nakagami',  'rayleigh',  'rice',  'maxwell']
dist_names = ['beta']
dist_names = ['levy_stable']

do_plot = 1
d_per_name = {}
pdf_fit_per_name = {}

for dist_name in dist_names:
    dist = getattr(scipy.stats, dist_name)
    params = dist.fit(y)
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]
    if arg:
        pdf_fitted = dist.pdf(binloc, *arg, loc=loc, scale=scale) * size
    else:
        pdf_fitted = dist.pdf(binloc, loc=loc, scale=scale) * size
    if do_plot and np.max(pdf_fitted) < np.max(binh) * 3:
        plt.plot(binloc, pdf_fitted, label=dist_name)
    plt.xlim(-6,20)
    pdf_fit_per_name[dist_name] = pdf_fitted
    
    discrep = np.sum(np.abs( binh-pdf_fitted[:-1] ) ) / len(binh)
    d_per_name[dist_name] = discrep
plt.legend(loc='upper right')
plt.show()

In [ ]:
arg

In [ ]:
help(sorted)

In [ ]:
stl = sorted( list( d_per_name.items() ), key = lambda x: x[1] )
stl_ext = [ (dn,v,pdf_fit_per_name[dn][0]) for  (dn, v) in stl]
display(stl_ext)

In [ ]:
list( set( ['exponpow', 'f', 'fatiguelife', 'fisk', 'foldcauchy', 'foldnorm', 
               'frechet_r', 'frechet_l', 'genlogistic', 'genpareto', 'genexpon', 
               'genextreme', 'gausshyper', 'gamma', 'gengamma', 'genhalflogistic', 
               'gilbrat', 'gompertz', 'gumbel_r', 'gumbel_l', 'halfcauchy'] ) - \
set( ['frechet_r',
 'genhalflogistic',
 'foldnorm',
 'gilbrat',
 'gompertz',
 'genpareto',
 'genexpon',
 'halfcauchy',
 'exponpow',
 'gamma',
 'frechet_l',
 'f',
 'gausshyper',
 'gumbel_l'] ) )

In [ ]:
list(list(zip(*stl))[0])[:10]

In [ ]:
list(list(zip(*stl))[0])[-14:]

In [ ]:
list(list(zip(*stl))[0])[-10:]

In [ ]:
len(binh), len(pdf_fitted)

In [ ]:
plt.plot(binh)
plt.plot(pdf_fitted[:-1])

In [ ]:
help(plt.subplots_adjust)